In [47]:
# Import libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [48]:
# load the datasets

books = pd.read_csv("c:/Users/Admin/books_cleaned.csv")
users = pd.read_csv("c:/Users/Admin/users_cleaned.csv")
ratings = pd.read_csv("c:/Users/Admin/ratings_cleaned.csv")

ratings_avg_count_nouser = pd.read_csv("c:/Users/Admin/ratings_avg_counts_nouser.csv")


print(books.shape)
print(users.shape)
print(ratings.shape)
print(ratings_avg_count_nouser.shape)

(271357, 7)
(168096, 4)
(383841, 3)
(433671, 5)


In [49]:
# Merge the "books" dataframe with ratings 

books_with_ratings = ratings_avg_count_nouser.merge(books,on='book_id')
books_with_ratings.head()

,Unnamed: 0_x,book_id,rating,avg_rating,rating_count,Unnamed: 0_y,title,authors,publication_year,Cover_image,mod_title
0,1,0155061224,5,5.00,5,225816,Rites of Passage,Judith Rae,2001,http://images.amazon.com/images/P/0155061224.0...,rites of passage
1,3,052165615X,3,3.00,3,246838,Help!: Level 1,Philip Prowse,1999,http://images.amazon.com/images/P/052165615X.0...,help level 1
2,4,0521795028,6,6.00,6,246839,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,http://images.amazon.com/images/P/0521795028.0...,the amsterdam connection level 4 cambridge en...
3,8,038550120X,7,7.58,614,9294,A Painted House,JOHN GRISHAM,2001,http://images.amazon.com/images/P/038550120X.0...,a painted house
4,48815,038550120X,10,7.58,614,9294,A Painted House,JOHN GRISHAM,2001,http://images.amazon.com/images/P/038550120X.0...,a painted house


In [50]:
# Drop useless columns

books_with_ratings.drop(['Unnamed: 0_x','rating','Unnamed: 0_y'],axis=1, inplace=True)

In [13]:
books_with_ratings.head()

,book_id,avg_rating,rating_count,Unnamed: 0_y,title,authors,publication_year,Cover_image,mod_title
0,0155061224,5.00,5,225816,Rites of Passage,Judith Rae,2001,http://images.amazon.com/images/P/0155061224.0...,rites of passage
1,052165615X,3.00,3,246838,Help!: Level 1,Philip Prowse,1999,http://images.amazon.com/images/P/052165615X.0...,help level 1
2,0521795028,6.00,6,246839,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,http://images.amazon.com/images/P/0521795028.0...,the amsterdam connection level 4 cambridge en...
3,038550120X,7.58,614,9294,A Painted House,JOHN GRISHAM,2001,http://images.amazon.com/images/P/038550120X.0...,a painted house
4,038550120X,7.58,614,9294,A Painted House,JOHN GRISHAM,2001,http://images.amazon.com/images/P/038550120X.0...,a painted house


In [58]:
# Drop title duplicates

books_with_ratings.drop_duplicates(subset='title', inplace=True)
books_with_ratings.head()

,book_id,avg_rating,rating_count,title,authors,publication_year,Cover_image,mod_title
0,0155061224,5.00,5,Rites of Passage,Judith Rae,2001,http://images.amazon.com/images/P/0155061224.0...,rites of passage
1,052165615X,3.00,3,Help!: Level 1,Philip Prowse,1999,http://images.amazon.com/images/P/052165615X.0...,help level 1
2,0521795028,6.00,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,http://images.amazon.com/images/P/0521795028.0...,the amsterdam connection level 4 cambridge en...
3,038550120X,7.58,614,A Painted House,JOHN GRISHAM,2001,http://images.amazon.com/images/P/038550120X.0...,a painted house
84,0060517794,8.00,240,Little Altars Everywhere,Rebecca Wells,2003,http://images.amazon.com/images/P/0060517794.0...,little altars everywhere


In [59]:
books_with_ratings.shape

(135566, 8)

In [60]:
books_with_ratings.to_csv('books_with_ratings_copy.csv',index = False)

# Popularity_based recommendation( the books with most ratings and higher average rating)

In [21]:
# List top 20 popular books

popular_books = books_with_ratings[books_with_ratings["rating_count"] > 200].sort_values(["avg_rating","rating_count"], ascending=False).head(10)


In [22]:
def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)

In [23]:
popular_books.head(10).style.format({'Cover_image': show_image})

,book_id,avg_rating,rating_count,Unnamed: 0_y,title,authors,publication_year,Cover_image,mod_title
156600,0439425220,9.870000,227,79370,Harry Potter and the Chamber of Secrets Postcard Book,J. K. Rowling,2002,,harry potter and the chamber of secrets postcard book
1309,0836220889,9.580000,230,15187,Calvin and Hobbes,Bill Watterson,1987,,calvin and hobbes
144967,0743454529,9.550000,210,30743,"My Sister's Keeper : A Novel (Picoult, Jodi)",Jodi Picoult,2004,,my sisters keeper a novel picoult jodi
46221,0060256672,9.400000,235,1101,Where the Sidewalk Ends : Poems and Drawings,Shel Silverstein,1974,,where the sidewalk ends poems and drawings
92925,0553274325,9.390000,216,10287,Johnny Got His Gun,Dalton Trumbo,1983,,johnny got his gun
6884,039480001X,9.310000,298,3776,The Cat in the Hat,Dr. Seuss,1957,,the cat in the hat
60180,0920668372,9.280000,232,4466,Love You Forever,Robert N. Munsch,1986,,love you forever
242900,1880418568,9.270000,204,69152,"Wolves of the Calla (The Dark Tower, Book 5)",Stephen King,2003,,wolves of the calla the dark tower book 5
59950,0836204387,9.260000,213,3921,The Calvin and Hobbes Tenth Anniversary Book,Bill Watterson,1995,,the calvin and hobbes tenth anniversary book
70846,0836218620,9.240000,231,15186,Weirdos From Another Planet!,Bill Watterson,1990,,weirdos from another planet


### Content based recommendation( Cosine Similarity)

In [66]:
# Import libraries

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

In [62]:
# TF-IDF(Term Frequency-Inverse Document Frequency) normalizes the title

vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(books_with_ratings["mod_title"])

In [70]:
# Define a funciton to show cover image

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)

In [75]:
# Define a function to search a book 

def search(query,vectorizer):
    # Clean and process the query
    processed = re.sub("[^a-zA-Z0-9 ]", "", query.lower())
    
    # Vectorize the query
    query_vec = vectorizer.transform([query])
    
    # Compute the Cosine Similarity
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    
    # Get book indices
    indices = np.argpartition(similarity, -10)[-10:]
    results = books_with_ratings.iloc[indices]
    
    # Get results
    return results.head(5).style.format({'Cover_image': show_image})

In [76]:
search("Little Altars Everywhere", vectorizer)

,book_id,avg_rating,rating_count,title,authors,publication_year,Cover_image,mod_title
364836,0500281505,9.000000,9,Beautiful Necessity: The Art and Meaning of Women's Altars,Kay Turner,1999,,beautiful necessity the art and meaning of womens altars
139628,0517704927,8.330000,25,Altars in the Street: A Neighborhood Fights to Survive,Melody Ermachild Chavis,1997,,altars in the street a neighborhood fights to survive
231853,1555972314,10.000000,10,Little,David Treuer,1995,,little
241266,0061042935,7.080000,85,Everywhere That Mary Went,Lisa Scottoline,1993,,everywhere that mary went
286064,1556704518,7.000000,7,In Search of Color Everywhere,E. Ethelbert Miller,1996,,in search of color everywhere
